In [1]:
import torch
from torch import nn
from torch.nn import functional as F

# model_name = "google/electra-large-discriminator"
model_name = "google/electra-base-discriminator"

In [2]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, ratings = eval_pred
    # predictions = np.argmax(predictions, axis=1)
    pred_pos = predictions > 5
    label_pos = ratings > 5
    # return accuracy.compute(predictions=pred_pos, references=label_pos)
    return {"accuracy": accuracy.compute(predictions=pred_pos, references=label_pos)["accuracy"],
            "rmse": np.mean((predictions - ratings)**2)**0.5}

# id2label = {0: "NEGATIVE", 1: "POSITIVE"}
# label2id = {"NEGATIVE": 0, "POSITIVE": 1}

bin c:\Users\Abstract\mambaforge\envs\sentenv2\lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


In [3]:
from datasets import load_dataset

imdb = load_dataset("./imdb_reg.py")
imdb.pop("unsupervised")
imdb, imdb["test"][0]

(DatasetDict({
     train: Dataset({
         features: ['text', 'label'],
         num_rows: 25000
     })
     test: Dataset({
         features: ['text', 'label'],
         num_rows: 25000
     })
 }),
 {'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care abou

In [4]:
imdb["test"][12500]

{'text': "Previous reviewer Claudio Carvalho gave a much better recap of the film's plot details than I could. What I recall mostly is that it was just so beautiful, in every sense - emotionally, visually, editorially - just gorgeous.<br /><br />If you like movies that are wonderful to look at, and also have emotional content to which that beauty is relevant, I think you will be glad to have seen this extraordinary and unusual work of art.<br /><br />On a scale of 1 to 10, I'd give it about an 8.75. The only reason I shy away from 9 is that it is a mood piece. If you are in the mood for a really artistic, very romantic film, then it's a 10. I definitely think it's a must-see, but none of us can be in that mood all the time, so, overall, 8.75.",
 'label': 10.0}

In [5]:
from transformers import DataCollatorWithPadding, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


def preprocess_function(examples):
    tokens = tokenizer(examples["text"], truncation=False)
    if type(tokens['input_ids'][0]) == list:
        for i in range(len(tokens['input_ids'])):
            if len(tokens['input_ids'][i]) > 512:
                tokens['input_ids'][i] = tokens['input_ids'][i][:129] + \
                    [102] + tokens['input_ids'][i][-382:]
                tokens['token_type_ids'][i] = [0]*512
                tokens['attention_mask'][i] = [1]*512
    elif len(tokens['input_ids']) > 512:
        tokens['input_ids'] = tokens['input_ids'][:129] + \
            [102] + tokens['input_ids'][-382:]
        tokens['token_type_ids'] = [0]*512
        tokens['attention_mask'] = [1]*512
    return tokens

In [6]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

In [7]:
tokenized_imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
})

In [8]:
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)

model

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [9]:
training_args = TrainingArguments(
    # output_dir="electra_large_imdb_reg_spliced_lr5e6",
    output_dir="electra_base_imdb_reg_spliced_fix10",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    warmup_steps=50,
    # torch_compile=True,
    # fp16=True,
    # load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

c:\Users\Abstract\mambaforge\envs\sentenv2\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/2343 [00:00<?, ?it/s]

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 34.0465, 'learning_rate': 2e-05, 'epoch': 0.06}
{'loss': 14.9392, 'learning_rate': 1.956389010030528e-05, 'epoch': 0.13}
{'loss': 7.6106, 'learning_rate': 1.9127780200610553e-05, 'epoch': 0.19}
{'loss': 4.3207, 'learning_rate': 1.869167030091583e-05, 'epoch': 0.26}
{'loss': 3.3331, 'learning_rate': 1.825556040122111e-05, 'epoch': 0.32}
{'loss': 2.9999, 'learning_rate': 1.7819450501526386e-05, 'epoch': 0.38}
{'loss': 2.9314, 'learning_rate': 1.738334060183166e-05, 'epoch': 0.45}
{'loss': 2.9101, 'learning_rate': 1.694723070213694e-05, 'epoch': 0.51}
{'loss': 2.5764, 'learning_rate': 1.651112080244222e-05, 'epoch': 0.58}
{'loss': 2.2788, 'learning_rate': 1.6075010902747493e-05, 'epoch': 0.64}
{'loss': 2.1878, 'learning_rate': 1.563890100305277e-05, 'epoch': 0.7}
{'loss': 2.0471, 'learning_rate': 1.5202791103358048e-05, 'epoch': 0.77}
{'loss': 2.2156, 'learning_rate': 1.4766681203663324e-05, 'epoch': 0.83}
{'loss': 2.4433, 'learning_rate': 1.4330571303968602e-05, 'epoch': 0.9}
{'

  0%|          | 0/391 [00:00<?, ?it/s]

...

  0%|          | 0/391 [00:00<?, ?it/s]

{'eval_loss': 1.9005815982818604, 'eval_accuracy': 0.95408, 'eval_rmse': 4.971129033816122, 'eval_runtime': 303.2016, 'eval_samples_per_second': 82.453, 'eval_steps_per_second': 1.29, 'epoch': 2.0}
{'loss': 1.4426, 'learning_rate': 6.480593109463585e-06, 'epoch': 2.05}
{'loss': 1.3537, 'learning_rate': 6.044483209768862e-06, 'epoch': 2.11}
{'loss': 1.2151, 'learning_rate': 5.608373310074139e-06, 'epoch': 2.18}
{'loss': 1.4597, 'learning_rate': 5.172263410379415e-06, 'epoch': 2.24}
{'loss': 1.206, 'learning_rate': 4.736153510684693e-06, 'epoch': 2.3}
{'loss': 1.3168, 'learning_rate': 4.30004361098997e-06, 'epoch': 2.37}
{'loss': 1.2819, 'learning_rate': 3.863933711295247e-06, 'epoch': 2.43}
{'loss': 1.3033, 'learning_rate': 3.4278238116005236e-06, 'epoch': 2.5}
{'loss': 1.0715, 'learning_rate': 2.9917139119058004e-06, 'epoch': 2.56}
{'loss': 1.3091, 'learning_rate': 2.5556040122110775e-06, 'epoch': 2.62}
{'loss': 1.3195, 'learning_rate': 2.1194941125163543e-06, 'epoch': 2.69}
{'loss': 1

  0%|          | 0/391 [00:00<?, ?it/s]

{'eval_loss': 1.8625514507293701, 'eval_accuracy': 0.95492, 'eval_rmse': 4.948355800483008, 'eval_runtime': 278.2431, 'eval_samples_per_second': 89.849, 'eval_steps_per_second': 1.405, 'epoch': 3.0}
{'train_runtime': 3152.6097, 'train_samples_per_second': 23.79, 'train_steps_per_second': 0.743, 'train_loss': 2.8849616610388242, 'epoch': 3.0}


TrainOutput(global_step=2343, training_loss=2.8849616610388242, metrics={'train_runtime': 3152.6097, 'train_samples_per_second': 23.79, 'train_steps_per_second': 0.743, 'train_loss': 2.8849616610388242, 'epoch': 3.0})

In [10]:
model(**(tokenizer("""When I was a kid I watched this many times over, and I remember whistling the "Happy Cat" song quite often. All the songs are great, and actually memorable, unlike many children's musicals, where the songs are just stuck in for no real reason. The scenes and costumes are lavish, and the acting is very well-done, which isn't surprising, considering the cast. Christopher Walken is very catlike, and doesn't need stupid make-up, or a cat costume for the viewer to believe he's a cat transformed to a human. And Jason Connery's so cute, as the shy and awkward miller's son, Corin, who falls in love with beautiful and the bold Princess Vera. This is a really fun, enjoyable, feature-length movie, where unlike most fairytales, the characters are given personalities. Some of my favourite parts are when Puss makes Corin pretend he's drowning; at the ball when everybody starts dancing a country dance, as it's "all the rage abroad"; when Walken is in the kitchen, dancing on the table (he's a pretty good dancer, too!); and when Vera tells Corin all the things she used to do when she was young, like pretending she was a miller's daughter. I'd recommend this film to children and parents alike, who love magic and fairytales. And it actually IS a movie you can watch together, as it won't drive adults up the wall.""", return_tensors="pt").to('cuda')))

SequenceClassifierOutput(loss=None, logits=tensor([[8.8437]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [20]:
model(**(tokenizer("It was a bit boring. Kinda drawn out but fine ig.", return_tensors="pt").to('cuda'))).logits

tensor([[6.1508]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [60]:
torch.cuda.empty_cache()

In [57]:
F.softmax(model(**(tokenizer("This was a fine movie, but I wouldn't watch it again to be honest, a bit too drawn out for my liking. My friends liked it though, so if you like detectives, you may want to watch it, if you have some spare time.", return_tensors="pt").to('cuda'))).logits, dim=1)

tensor([[0.3959, 0.6041]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [55]:
F.softmax(model(**(tokenizer("I was absolutely extatic when I left the cinema! It was such a relief when I knew that it was over and I won't have to watch this joke of a movie again! Honestly would go to this movie again just to leave as soon as it starts, lol", return_tensors="pt").to('cuda'))).logits, dim=1)

tensor([[0.9930, 0.0070]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [20]:
model.forward(**(tokenizer('hello', return_tensors="pt").to('cuda')))

SequenceClassifierOutput(loss=None, logits=tensor([[-0.2006,  0.1987]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)